In [9]:
import pandas as pd
import datetime

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [10]:
input_price = pd.read_excel("daily_trading_info_2014-2018.xlsx", sheet_name = None)

In [11]:
stock_index = []

stock_index = list(input_price.keys())[0:]

In [12]:
results = pd.read_excel("strategy_results_train.xlsx", sheet_name = "strategy_results_train")

In [13]:
import copy

In [14]:
dataset = copy.deepcopy(results)

In [15]:
def ma(input_price, trade_company, trade_date_datetime, ma_length):
    df = input_price[trade_company]
    date_index = 0
    for i in range(0, df.shape[0]):
        if(df.loc[i][1] >= trade_date_datetime):
#             print(i, df.iloc[i][1])
            date_index = i
            break
    ma = 0
    ma_length = 3
    for reverse in range(0, ma_length):
        ma = ma + df.iloc[date_index-reverse][3]
    ma = ma/ ma_length
    return ma

In [16]:
temp = []
for i in range(0, dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = dataset.iloc[i][0].split("_")[0]
    trade_date    = dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma(input_price, trade_company, trade_date_datetime, ma_length = 3))
#     print(i, trade_company, trade_date, trade_date_datetime)

0 2020-06-21 09:25:29.431061
1000 2020-06-21 09:32:44.330682
2000 2020-06-21 09:40:51.902075
3000 2020-06-21 09:49:24.982350
4000 2020-06-21 09:57:59.942937
5000 2020-06-21 10:06:08.153764
6000 2020-06-21 10:14:15.508560
7000 2020-06-21 10:21:09.361170
8000 2020-06-21 10:28:19.389582
9000 2020-06-21 10:35:44.251033
10000 2020-06-21 10:42:37.007678
11000 2020-06-21 10:49:38.451259
12000 2020-06-21 10:58:12.116821
13000 2020-06-21 11:06:20.487057
14000 2020-06-21 11:14:47.638702
15000 2020-06-21 11:22:26.813545
16000 2020-06-21 11:30:46.825321
17000 2020-06-21 11:38:28.103568
18000 2020-06-21 11:45:27.145047
19000 2020-06-21 11:52:51.742728
20000 2020-06-21 12:00:30.192368
21000 2020-06-21 12:08:02.689969
22000 2020-06-21 12:15:09.821201
23000 2020-06-21 12:23:00.942793
24000 2020-06-21 12:30:58.083310
25000 2020-06-21 12:38:16.208368
26000 2020-06-21 12:45:51.964179
27000 2020-06-21 12:52:17.495779
28000 2020-06-21 13:00:19.897308
29000 2020-06-21 13:08:06.073012
30000 2020-06-21 13:16:

In [17]:
pd.DataFrame(temp, columns=["ma"]).head()
temp = pd.DataFrame(temp, columns=["ma"])

In [18]:
temp

,ma
0,22.080000
1,22.980000
2,22.603333
3,26.633334
4,33.919999
...,...
31942,59.093333
31943,55.533333
31944,57.226667
31945,47.346668


In [19]:
dataset.shape

(31947, 2)

In [20]:
result = pd.concat([dataset, pd.DataFrame(temp).reindex(dataset.index)], axis=1)
result.head()

,id,label,ma
0,SHSE.600085_201501,0,22.080000
1,SHSE.600085_201502,1,22.980000
2,SHSE.600085_201503,1,22.603333
3,SHSE.600085_201504,1,26.633334
4,SHSE.600085_201505,1,33.919999


In [21]:
result.to_excel("train_output.xlsx")  

In [22]:
y = pd.DataFrame(result["label"])
X = pd.DataFrame(result["ma"])

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [24]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
print("train f1: ", f1_score(y_train, clf.predict(X_train)))
print("valid f1: ", f1_score(y_valid, clf.predict(X_valid)))
print("train accuracy: ", accuracy_score(y_train, clf.predict(X_train)))
print("valid accuracy: ", accuracy_score(y_valid, clf.predict(X_valid)))
print("train recall_score: ", recall_score(y_train, clf.predict(X_train)))
print("valid recall_score: ", recall_score(y_valid, clf.predict(X_valid)))
print("train precision_score: ", precision_score(y_train, clf.predict(X_train)))
print("valid precision_score: ", precision_score(y_valid, clf.predict(X_valid)))
fpr, tpr, thresholds = metrics.roc_curve(y_train.to_numpy().flatten(), clf.predict(X_train).flatten(), pos_label=1)
print("train auc: ", metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(y_valid.to_numpy().flatten(), clf.predict(X_valid).flatten(), pos_label=1)
print("valid auc: ", metrics.auc(fpr, tpr))

train f1:  0.06455646737006644
valid f1:  0.07609696225836148
train accuracy:  0.5316742966701882
valid accuracy:  0.5287949921752738
train recall_score:  0.03454621497281472
valid recall_score:  0.04068241469816273
train precision_score:  0.49166666666666664
valid precision_score:  0.5876777251184834
train auc:  0.5015768863424579
valid auc:  0.5073250493598533


In [26]:
y_train = pd.DataFrame(result["label"])
X_train = pd.DataFrame(result["ma"])
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
test_input_price = pd.read_excel("daily_trading_info_2019-2020.xlsx", sheet_name = None)

In [28]:
test_stock_index = []
test_stock_index = list(test_input_price.keys())[0:]

In [29]:
test_results = pd.read_csv("sampleSubmission.csv")

In [30]:
test_results.head()

,id,label
0,SHSE.600085_201901,0
1,SHSE.600085_201902,0
2,SHSE.600085_201903,0
3,SHSE.600085_201904,0
4,SHSE.600085_201905,0


In [31]:
import copy

In [32]:
test_dataset = copy.deepcopy(test_results)

In [33]:
temp = []
for i in range(0, test_dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = test_dataset.iloc[i][0].split("_")[0]
    trade_date    = test_dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma(test_input_price, trade_company, trade_date_datetime, ma_length = 3))
#     print(i, trade_company, trade_date)

0 2020-06-21 13:31:55.641119
1000 2020-06-21 13:33:52.439135
2000 2020-06-21 13:35:56.096978
3000 2020-06-21 13:37:59.679542
4000 2020-06-21 13:40:06.945289
5000 2020-06-21 13:42:10.541846
6000 2020-06-21 13:44:17.947257
7000 2020-06-21 13:46:22.251270
8000 2020-06-21 13:48:27.213098
9000 2020-06-21 13:50:31.556760
10000 2020-06-21 13:52:35.520167
11000 2020-06-21 13:54:39.373966


In [34]:
len(temp)

11363

In [35]:
test_dataset.shape[0]

11363

In [36]:
temp = pd.DataFrame(temp, columns=["ma"])
test_result = pd.concat([test_dataset, temp], axis=1)
test_result.head()

,id,label,ma
0,SHSE.600085_201901,0,26.230000
1,SHSE.600085_201902,0,26.676667
2,SHSE.600085_201903,0,30.040000
3,SHSE.600085_201904,0,30.116667
4,SHSE.600085_201905,0,31.499999


In [37]:
test_result.to_excel("test_output.xlsx")  

In [38]:
test_X = pd.DataFrame(test_result["ma"])

In [39]:
test_X.head(10)

,ma
0,26.230000
1,26.676667
2,30.040000
3,30.116667
4,31.499999
5,29.606667
6,29.416667
7,28.323334
8,28.273333
9,27.160000


In [40]:
test_label=pd.DataFrame(clf.predict(test_X),columns=["label"]) 

In [41]:
test_data=pd.DataFrame(test_dataset)

In [42]:
test= pd.concat([test_data,test_label], axis=1).to_excel("test_output.xlsx")  

In [43]:
pd.read_excel("test_output.xlsx")

,id,label
0,SHSE.600085_201901,0
1,SHSE.600085_201902,0
2,SHSE.600085_201903,0
3,SHSE.600085_201904,0
4,SHSE.600085_201905,0
...,...,...
11358,SHSE.603939_201911,1
11359,SHSE.603939_201912,1
11360,SHSE.603939_202001,1
11361,SHSE.603939_202002,1
